In [1]:

from jailbreaks.methods.model import DiffInMeans

%load_ext autoreload
%autoreload 2

/Users/joro/src/kth/thesis/jailbreaks/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
d = DiffInMeans()

2025-03-31 17:39:57,415 - DiffInMeans - WARNING - No saved hooks found at jailbreaks/checkpoints/diff-in-means.pkl


In [5]:
d.model2hooks

{}

In [6]:
model_path = "Qwen/Qwen2-0.5B-Instruct"
model_path = "Qwen/Qwen-1_8B-chat"

In [7]:
import requests
import pandas as pd
import io
from datasets import load_dataset

def get_advbench_instructions():
    url = 'https://raw.githubusercontent.com/llm-attacks/llm-attacks/main/data/advbench/harmful_behaviors.csv'
    response = requests.get(url)

    dataset = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
    instructions = dataset['goal'].tolist()
    return instructions

def get_harmless_instructions():
    hf_path = 'tatsu-lab/alpaca'
    dataset = load_dataset(hf_path)

    # filter for instructions that do not have inputs
    instructions = []
    for i in range(len(dataset['train'])):
        if dataset['train'][i]['input'].strip() == '':
            instructions.append(dataset['train'][i]['instruction'])

    return instructions

In [8]:
harmful_prompts = get_advbench_instructions()
harmless_prompts = get_harmless_instructions()


In [ ]:
d.fit(model_path, harmful_prompts[:5], harmless_prompts[:5])

2025-03-31 17:40:28,315 - DiffInMeans - INFO - Loading model Qwen/Qwen-1_8B-chat
2025-03-31 17:40:28,316 - root - WARNING - float16 models may not work on CPU. Consider using a GPU or bfloat16.
2025-03-31 17:40:29,180 - transformers_modules.Qwen.Qwen-1_8B-Chat.1d0f68de57b88cfde81f3c3e537f24464d889081.modeling_qwen - WARNING - Try importing flash-attention for faster inference...
2025-03-31 17:40:29,181 - transformers_modules.Qwen.Qwen-1_8B-Chat.1d0f68de57b88cfde81f3c3e537f24464d889081.modeling_qwen - WARNING - Warning: import flash_attn rotary fail, please install FlashAttention rotary to get higher efficiency https://github.com/Dao-AILab/flash-attention/tree/main/csrc/rotary
2025-03-31 17:40:29,182 - transformers_modules.Qwen.Qwen-1_8B-Chat.1d0f68de57b88cfde81f3c3e537f24464d889081.modeling_qwen - WARNING - Warning: import flash_attn rms_norm fail, please install FlashAttention layer_norm to get higher efficiency https://github.com/Dao-AILab/flash-attention/tree/main/csrc/layer_norm
20

Loaded pretrained model Qwen/Qwen-1_8B-chat into HookedTransformer


2025-03-31 17:40:39,211 - DiffInMeans - INFO - Running harmful prompts with cache
2025-03-31 17:47:21,345 - DiffInMeans - INFO - Running harmless prompts with cache
